In [1]:
import yfinance as yf
import pandas
import numpy
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
import datetime

tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'INTC', 'AMD', 'NVDA', 'F', 'TSLA', 'JPM', 'MS', 'VOO']

def get_monthly_data_from_yf(ticker, start_date, end_date):
    return yf.download(ticker, start_date, end_date, interval='1mo', progress=False).dropna()

# Get data for all tickers 2012 - 2021
data_ten_years = {}
for ticker in tickers:
    data_ten_years[ticker] = get_monthly_data_from_yf(ticker, '2012-01-01', '2022-01-01')
print('All data loaded')

Using TensorFlow backend.


All data loaded


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def predict_next_month_close_price(dataset, ticker):
    # fix random seed for reproducibility
    numpy.random.seed(7)
    dataframe = dataset[ticker]
    dataframe = dataframe.loc[:, dataframe.columns.isin(['Close'])]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    #print("Train data size =", len(train), "Test data size =", len(test))
    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    # print test arrays X and Y 
    #for i in range(len(testX)):
    #    print(testX[i], testY[i])
    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    #print("LSTM network started...", "Train data size =", len(train), "Test data size =", len(test))
    start_time_ms = time.time() * 1000
    model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)        # verbose=2
    end_time_ms = time.time() * 1000
    #print("LSTM network finished. Time spent = %.2f seconds." % ((end_time_ms - start_time_ms) / 1000))
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    #trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    #print('Train Score: %.2f RMSE' % (trainScore))
    #testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    #print('Test Score: %.2f RMSE' % (testScore))
    # MAKE PREDICTION FOR NEXT MONTH
    last_real_data = scaler.inverse_transform(test)
    last_real_data = last_real_data[len(last_real_data)-1:]
    #print('last real data =', last_real_data)
    last_real_data_scaled = scaler.transform(last_real_data)
    #print('last real data scaled =', last_real_data_scaled)
    featureX = last_real_data_scaled[(len(last_real_data_scaled)-1):(len(last_real_data_scaled)),:]          # last value as array
    #print('featureX =', featureX.tolist())
    featureX = numpy.reshape(featureX, (featureX.shape[0], 1, featureX.shape[1]))
    prediction_next_month = model.predict(featureX)
    last_real_data_scaled = numpy.concatenate((last_real_data_scaled, prediction_next_month), axis=0)
    #print('last real data + one prediction scaled =', last_real_data_scaled.tolist())
    #print('prediction for next month scaled =', prediction_next_month)
    #print('prdiction for next month = ', scaler.inverse_transform(prediction_next_month))
    next_month_inversed = scaler.inverse_transform(prediction_next_month)[0][0]
    #print(ticker+":", 'Last known Close price =', last_real_data[0][0], '| Next month Close prise =', next_month_inversed)
    return next_month_inversed

def weighted_next_month_close_price(data, ticker):
    prediction_1 = predict_next_month_close_price(data, ticker)
    prediction_2 = predict_next_month_close_price(data, ticker)
    prediction_3 = predict_next_month_close_price(data, ticker)
    return (prediction_1 + prediction_2 + prediction_3) / 3

In [3]:
predictions_next_month_close_price = {}
for ticker in tickers:
    next_month_close_price = weighted_next_month_close_price(data_ten_years, ticker)
    predictions_next_month_close_price[ticker] = next_month_close_price
    print(ticker, "WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION =", next_month_close_price)
print(predictions_next_month_close_price)

AAPL WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 164.60960896809897
GOOG WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 142.93388875325522
MSFT WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 311.9177652994792
AMZN WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 167.2767537434896
INTC WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 51.61640421549479
AMD WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 148.98928833007812
NVDA WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 268.7667643229167
F WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 20.1096674601237
TSLA WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 275.5469156901042
JPM WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 153.43032836914062
MS WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 94.70916748046875
VOO WEIGHTED NEXT MONTH CLOSE PRICE PREDICTION = 421.8673502604167
{'AAPL': 164.60960896809897, 'GOOG': 142.93388875325522, 'MSFT': 311.9177652994792, 'AMZN': 167.2767537434896, 'INTC': 51.61640421549479, 'AMD': 148.98928833007812, 'NVDA': 268.766764322916

In [4]:
data_for_portfolio = yf.download(tickers, start='2020-01-01', end='2022-01-01', interval='1mo')['Close'].dropna()

d = datetime.datetime.strptime("01/01/2022","%d/%m/%Y")
data_for_portfolio_with_prediction = data_for_portfolio.append(pandas.DataFrame(index=[d]))
for ticker in tickers:
    data_for_portfolio_with_prediction.loc[d, ticker] = predictions_next_month_close_price[ticker]

data_for_portfolio.tail()

[*********************100%***********************]  12 of 12 completed


,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,TSLA,VOO
Date,,,,,,,,,,,,
2021-08-01,151.830002,110.720001,173.539505,13.030000,145.462006,54.060001,159.949997,104.430000,301.880005,223.850006,245.240005,415.049988
2021-09-01,141.500000,102.900002,164.251999,14.160000,133.265503,53.279999,163.690002,97.309998,281.920013,207.160004,258.493347,394.399994
2021-10-01,149.800003,120.230003,168.621506,17.080000,148.270493,49.000000,169.889999,102.779999,331.619995,255.669998,371.333344,422.160004
2021-11-01,165.300003,158.369995,175.353500,19.190001,142.451996,49.200001,158.830002,94.820000,330.589996,326.760010,381.586670,419.059998
2021-12-01,177.570007,143.899994,166.716995,20.770000,144.679504,51.500000,158.350006,98.160004,336.320007,294.109985,352.260010,436.570007


In [5]:
data_for_portfolio_with_prediction.tail()

,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,TSLA,VOO
2021-09-01,141.500000,102.900002,164.251999,14.160000,133.265503,53.279999,163.690002,97.309998,281.920013,207.160004,258.493347,394.399994
2021-10-01,149.800003,120.230003,168.621506,17.080000,148.270493,49.000000,169.889999,102.779999,331.619995,255.669998,371.333344,422.160004
2021-11-01,165.300003,158.369995,175.353500,19.190001,142.451996,49.200001,158.830002,94.820000,330.589996,326.760010,381.586670,419.059998
2021-12-01,177.570007,143.899994,166.716995,20.770000,144.679504,51.500000,158.350006,98.160004,336.320007,294.109985,352.260010,436.570007
2022-01-01,164.663239,149.008341,167.273560,20.109227,142.937673,51.615300,153.432699,94.707672,311.950358,268.766744,275.527486,421.863444


In [6]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

### Portfolio based only on real data
mu = mean_historical_return(data_for_portfolio, frequency=12)
S = CovarianceShrinkage(data_for_portfolio, frequency=12).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
#weights = ef.max_sharpe()
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
print('- - -')
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.0), ('AMD', 0.00892), ('AMZN', 0.14382), ('F', 0.03027), ('GOOG', 0.05891), ('INTC', 0.23247), ('JPM', 0.10737), ('MS', 0.0), ('MSFT', 0.17157), ('NVDA', 0.09703), ('TSLA', 0.0), ('VOO', 0.14964)])
- - -
Expected annual return: 31.3%
Annual volatility: 17.1%
Sharpe Ratio: 1.72


(0.31343964875617136, 0.17050669365870896, 1.7209860942088788)

In [7]:
### Portfolio based on real data + next month prediction
mu = mean_historical_return(data_for_portfolio_with_prediction, frequency=12)
S = CovarianceShrinkage(data_for_portfolio_with_prediction, frequency=12).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
#weights = ef.max_sharpe()
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
print('- - -')
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.0), ('AMD', 0.02006), ('AMZN', 0.15416), ('F', 0.02581), ('GOOG', 0.06404), ('INTC', 0.25192), ('JPM', 0.10441), ('MS', 0.0), ('MSFT', 0.15358), ('NVDA', 0.07665), ('TSLA', 0.0), ('VOO', 0.14938)])
- - -
Expected annual return: 25.3%
Annual volatility: 17.1%
Sharpe Ratio: 1.36


(0.2525851267808363, 0.17076056755550972, 1.3620540743707066)